In [ ]:
import config as cfg
import network
import vctoolkit as vc
import skeletons as sk
import math_np
import torch
import numpy as np

In [ ]:
# you need to set model path here
model_path = '../../../ckpt/iknet_778_v1/14000.pth'
model = network.IKNetV1(cfg.MANO_VERTS * 3, sk.MANOHand.n_joints * 6, shape_dim=10, depth=4, width=2048).to(cfg.DEVICE)
model.load_state_dict(torch.load(model_path, map_location=torch.device(cfg.DEVICE))['model'])
model.eval()

In [ ]:
# here we load an obj file as example
# you can use your own vertices of shape (778, 3)
# make sure the unit is decimeter, and the mesh should be put in the origin
obj_path = 'output/mano_dataloader/mesh_0.obj'
v, _ = vc.load(obj_path)
v.shape

In [ ]:
# run the model
with torch.no_grad():
  prediction = model(torch.from_numpy(np.reshape(v.astype(np.float32), [1, -1])).to(cfg.DEVICE))
prediction = {k: v[0].detach().cpu().numpy() for k, v in prediction.items()}

In [ ]:
# the pose difinition of IKNet is a bit different from the original mano
# and we convert it back to the original format
def recover_original_mano_pose(pose):
  pose = math_np.convert(np.reshape(pose, [-1, 6]), 'rot6d', 'axangle')
  mano_pose = np.zeros([16, 3])
  mano_pose[0] = pose[4] # use M0 to represent palm
  for finger in 'TIMRL':
    for knuckle in [0, 1, 2]:
      mano_pose[sk.MANOHand.labels.index(finger + str(knuckle))] = \
        pose[sk.MANOHand.labels.index(finger + str(knuckle + 1))]
  return mano_pose

In [ ]:
prediction['pose'] = recover_original_mano_pose(prediction['pose'])

In [ ]:
vc.examine_dict(prediction)